# 数据流水线技术

In [1]:
import functools

import numpy as np
import pandas as pd 
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

In [3]:
# 用来显示一下数据的内容。并非使用pandas进行数据读取。
df = pd.read_csv(train_file_path)
df.head()

,survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,1,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,1,female,35.0,1,0,53.1000,First,C,Southampton,n
4,0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y


In [4]:
# 1 从文件中直接读取csv数据并创建dataset
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

LABEL_COLUMN = 'survived'
LABELS = [0, 1]

def get_dataset(file_path):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=12, # 为了示例更容易展示，手动设置较小的值
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [5]:
# 1 加载数据，并查看已经加载的第一个数据。
examples, labels = next(iter(raw_train_data)) # 第一个批次
print("EXAMPLES: \n", examples, "\n")
print("LABELS: \n", labels)

EXAMPLES: 
 OrderedDict([('sex', <tf.Tensor: shape=(12,), dtype=string, numpy=
array([b'male', b'female', b'female', b'male', b'male', b'male',
       b'female', b'male', b'female', b'male', b'male', b'female'],
      dtype=object)>), ('age', <tf.Tensor: shape=(12,), dtype=float32, numpy=
array([34., 40., 28.,  2., 56., 34., 38., 20., 28., 28., 37., 44.],
      dtype=float32)>), ('n_siblings_spouses', <tf.Tensor: shape=(12,), dtype=int32, numpy=array([0, 1, 0, 4, 0, 1, 1, 0, 0, 0, 0, 0])>), ('parch', <tf.Tensor: shape=(12,), dtype=int32, numpy=array([0, 1, 0, 1, 0, 0, 5, 0, 0, 0, 1, 0])>), ('fare', <tf.Tensor: shape=(12,), dtype=float32, numpy=
array([ 6.4958, 39.    , 79.2   , 39.6875, 26.55  , 26.    , 31.3875,
        7.8542,  7.75  , 39.6   , 29.7   , 27.7208], dtype=float32)>), ('class', <tf.Tensor: shape=(12,), dtype=string, numpy=
array([b'Third', b'Second', b'First', b'Third', b'First', b'Second',
       b'Third', b'Third', b'Third', b'First', b'First', b'First'],
      dtype=o

In [6]:
# 2 对数据预处理——参考结构化数据处理的部分。文本类数据转换成数字
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [7]:
# 2 数据预处理——特征数据，数值数据标准化
def process_continuous_data(mean, data):
  # 标准化数据
  data = tf.cast(data, tf.float32) * 1/(2*mean)
  return tf.reshape(data, [-1, 1])


MEANS = {
    'age' : 29.631308,
    'n_siblings_spouses' : 0.545455,
    'parch' : 0.379585,
    'fare' : 34.385399
}

numerical_columns = []

for feature in MEANS.keys():
  num_col = tf.feature_column.numeric_column(feature, normalizer_fn=functools.partial(process_continuous_data, MEANS[feature]))
  numerical_columns.append(num_col)

In [8]:
# 2 数据处理——创建数据预处理层
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numerical_columns)

In [9]:
# 3 构建模型
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])
    

In [10]:
train_data = raw_train_data.shuffle(500)
test_data = raw_test_data
model.fit(train_data, epochs=20)

Epoch 1/20
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
53/53 [==============================] - 1s 2ms/step - loss: 0.5951 - accuracy: 0.6944
Epoch 2/20
53/53 [==============================] - 0s 1ms/step - loss: 0.4517 - accuracy: 0.7968
Epoch 3/20
53/53 [==============================] - 0s 1ms/step - loss: 0.4124 - accuracy: 0.8221
Epoch 4/20
53/53 [==============================] - 0s 2ms/step - loss: 0.3807 - accuracy: 0.8331
Epoch 5/20
53/53 [==============================] - 0s 1ms/step - loss: 0.4360 - accuracy: 0.8041
Epoch 6/20
53/53 [==============================] - 0s 1ms/step - loss: 0.3664 - accuracy: 0.8444
Epoch 7/20
53/53 [==============================] - 0s 1ms/step - loss: 0.3733 - accuracy: 0.8437
Epoch 8/20
53/53 [==============================] - 0s 1ms/step - loss: 0.3783 - accuracy: 0.8281
Epoch 9/20
53/53 [==============================] - 0s 1ms/step - loss: 0.4091 - accuracy: 0.8267
Epoch 10

In [11]:
# 4 验证模型
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

Consider rewriting this model with the Functional API.
22/22 [==============================] - 0s 1ms/step - loss: 0.4477 - accuracy: 0.8182


Test Loss 0.4477050304412842, Test Accuracy 0.8181818127632141


In [12]:
predictions = model.predict(test_data)

# 显示部分结果
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))

Consider rewriting this model with the Functional API.
Predicted survival: 12.24%  | Actual outcome:  SURVIVED
Predicted survival: 10.38%  | Actual outcome:  SURVIVED
Predicted survival: 13.67%  | Actual outcome:  DIED
Predicted survival: 64.86%  | Actual outcome:  SURVIVED
Predicted survival: 4.28%  | Actual outcome:  DIED
Predicted survival: 27.77%  | Actual outcome:  DIED
Predicted survival: 13.21%  | Actual outcome:  DIED
Predicted survival: 8.61%  | Actual outcome:  SURVIVED
Predicted survival: 9.82%  | Actual outcome:  SURVIVED
Predicted survival: 60.03%  | Actual outcome:  SURVIVED
